In [1]:
import trimesh

# Load the mesh
tokyo_mesh = trimesh.load("a_3DMap_Data/buildings.obj")

# Show some infos
print(tokyo_mesh)

# Simple matplotlib 3D plot
#tokyo_mesh.show()

<trimesh.Trimesh(vertices.shape=(2564689, 3), faces.shape=(3170984, 3))>


In [2]:
import trimesh
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# Load mesh
vertices = tokyo_mesh.vertices.copy()
faces = tokyo_mesh.faces

# Shift so xmin=ymin=0
xmin, ymin, zmin = vertices.min(axis=0)
xmax, ymax, zmax = vertices.max(axis=0)

shifted_vertices = vertices - np.array([xmin, ymin, 0])

# Scale X/Y/Z
scale_x = 1000.0 / shifted_vertices[:,0].max()
scale_y = 1000.0 / shifted_vertices[:,1].max()
scale_z = 100.0 / shifted_vertices[:,2].max()
scaled_vertices = shifted_vertices.copy()
scaled_vertices[:,0] *= scale_x
scaled_vertices[:,1] *= scale_y
scaled_vertices[:,2] *= scale_z

In [3]:
print(xmin, ymin, zmin)
print(xmax, ymax, zmax)

384592.1875 3944795.0 0.0
391455.15625 3952352.5 264.600006


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Folder containing CSV files
csv_folder = Path("b_training_propagation_loss/7GHz")

# Initialize storage
tx_locations_raw = []   # TX before scaling/shift
rx_locations_raw = []   # RX before scaling/shift

print("Reading CSV files...")
for csv_file in csv_folder.glob("*.csv"):
    print(f"  Processing {csv_file.name}")
    df = pd.read_csv(csv_file)

    # TX: first row (x, y, z)
    tx = df.iloc[0, [0, 1, 2]].values
    tx_locations_raw.append(tx)

    # RX: all rows (x, y, z)
    rx_all = df.iloc[:, [6, 7, 8]]
    rx_locations_raw.append(rx_all.to_numpy())

# Combine all RX into one big array
rx_locations_raw = np.vstack(rx_locations_raw)

# Drop duplicates
rx_locations_raw = (
    pd.DataFrame(rx_locations_raw, columns=["x", "y", "z"])
    .drop_duplicates()
    .to_numpy()
)

# Convert TX list to array
tx_locations_raw = np.array(tx_locations_raw)

print(f"Total TX found: {len(tx_locations_raw)}")
print(f"Unique RX found: {len(rx_locations_raw)}")

# ------------------------------------------
# Shift and scale using mesh alignment
# ------------------------------------------

print("Shifting and scaling coordinates...")

# Shift x,y so that xmin=ymin=0 (from mesh precomputation)
tx_locations_shifted = tx_locations_raw.copy()
rx_locations_shifted = rx_locations_raw.copy()

tx_locations_shifted[:, 0] -= xmin
tx_locations_shifted[:, 1] -= ymin
rx_locations_shifted[:, 0] -= xmin
rx_locations_shifted[:, 1] -= ymin

# Scale (using same mesh scaling factors)
tx_locations_scaled = tx_locations_shifted.copy()
rx_locations_scaled = rx_locations_shifted.copy()

tx_locations_scaled[:, 0] *= scale_x
tx_locations_scaled[:, 1] *= scale_y
tx_locations_scaled[:, 2] *= scale_z

rx_locations_scaled[:, 0] *= scale_x
rx_locations_scaled[:, 1] *= scale_y
rx_locations_scaled[:, 2] *= scale_z

print("Coordinate transformation complete ✅")

Reading CSV files...
  Processing 7GHz_Tx_1.csv
  Processing 7GHz_Tx_10.csv
  Processing 7GHz_Tx_11.csv
  Processing 7GHz_Tx_13.csv
  Processing 7GHz_Tx_15.csv
  Processing 7GHz_Tx_16.csv
  Processing 7GHz_Tx_17.csv
  Processing 7GHz_Tx_18.csv
  Processing 7GHz_Tx_19.csv
  Processing 7GHz_Tx_2.csv
  Processing 7GHz_Tx_3.csv
  Processing 7GHz_Tx_4.csv
  Processing 7GHz_Tx_6.csv
  Processing 7GHz_Tx_7.csv
  Processing 7GHz_Tx_8.csv
Total TX found: 15
Unique RX found: 182623
Shifting and scaling coordinates...
Coordinate transformation complete ✅


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Folder containing CSV files (test points)
csv_folder = Path("c_evaluation_propagation_loss/7GHz")

# Store TX and RX test point locations
tx_test_raw = []
rx_test_raw = []  # plain list

print("Reading CSV files...")
# Loop over CSV files
for csv_file in csv_folder.glob("*.csv"):
    print(f"  Processing {csv_file.name}")
    df = pd.read_csv(csv_file)

    # TX: first row, columns 0,1,2 (x,y,z)
    tx = df.iloc[0, [0, 1, 2]].values
    tx_test_raw.append(tx)

    # RX: all rows, columns 6,7,8 (x,y,z)
    rx_all = df.iloc[:, [6, 7, 8]]
    rx_test_raw.append(rx_all.to_numpy())

# Combine all RX into one big array
rx_test_raw = np.vstack(rx_test_raw)

# Drop duplicates
rx_test_raw = (
    pd.DataFrame(rx_test_raw, columns=["x", "y", "z"])
    .drop_duplicates()
    .to_numpy()
)

print(f"Total TX test points found: {len(tx_test_raw)}")
print(f"Unique RX test points found: {len(rx_test_raw)}")

# Convert TX to numpy
tx_test_raw = np.array(tx_test_raw)

print("Shifting coordinates...")
# Shifted coordinates
tx_test_shifted = tx_test_raw.copy()
rx_test_shifted = rx_test_raw.copy()

tx_test_shifted[:, 0] -= xmin
tx_test_shifted[:, 1] -= ymin
rx_test_shifted[:, 0] -= xmin
rx_test_shifted[:, 1] -= ymin

print("Scaling coordinates...")
# Scaled coordinates
tx_test_scaled = tx_test_shifted.copy()
rx_test_scaled = rx_test_shifted.copy()

tx_test_scaled[:, 0] *= scale_x
tx_test_scaled[:, 1] *= scale_y
tx_test_scaled[:, 2] *= scale_z

rx_test_scaled[:, 0] *= scale_x
rx_test_scaled[:, 1] *= scale_y
rx_test_scaled[:, 2] *= scale_z

Reading CSV files...
  Processing 7GHz_Tx_12.csv
  Processing 7GHz_Tx_14.csv
  Processing 7GHz_Tx_20.csv
  Processing 7GHz_Tx_5.csv
  Processing 7GHz_Tx_9.csv
Total TX test points found: 5
Unique RX test points found: 94453
Shifting coordinates...
Scaling coordinates...


In [6]:
import numpy as np

# --- Full city mesh (shifted) ---
city_vertices_full = shifted_vertices
city_faces_full = faces

print(f"Original mesh has {len(city_vertices_full)} vertices and {len(city_faces_full)} faces.")

# Compute bounding box of the full city mesh
city_bbox_min = city_vertices_full.min(axis=0)
city_bbox_max = city_vertices_full.max(axis=0)

print("Full city mesh bounding box:")
print("Min:", city_bbox_min)
print("Max:", city_bbox_max)

# --- Compute bounding box of all Rx and Tx points ---
all_points = np.vstack([rx_locations_shifted, tx_locations_shifted])
rx_tx_bbox_min = all_points.min(axis=0)
rx_tx_bbox_max = all_points.max(axis=0)

print("Rx/Tx bounding box (XY only):")
print("Min (XY):", rx_tx_bbox_min[:2])
print("Max (XY):", rx_tx_bbox_max[:2])

# --- For each face, get its vertices ---
face_vertices_full = city_vertices_full[city_faces_full]  # shape: (num_faces, 3, 3)

# --- Boolean masks: inside and outside Rx–Tx XY bounding box ---
mask_inside = np.any(
    (face_vertices_full[:, :, 0] >= rx_tx_bbox_min[0]) &
    (face_vertices_full[:, :, 0] <= rx_tx_bbox_max[0]) &
    (face_vertices_full[:, :, 1] >= rx_tx_bbox_min[1]) &
    (face_vertices_full[:, :, 1] <= rx_tx_bbox_max[1]),
    axis=1
)
mask_outside = ~mask_inside

# ----------------- INSIDE MESH -----------------
city_faces_inside = city_faces_full[mask_inside]
unique_vertices_inside = np.unique(city_faces_inside)
city_vertices_inside = city_vertices_full[unique_vertices_inside]

# Reindex
vertex_reindex_inside = {old: new for new, old in enumerate(unique_vertices_inside)}
city_faces_inside_reindexed = np.vectorize(vertex_reindex_inside.get)(city_faces_inside)

print(f"Inside mesh has {len(city_vertices_inside)} vertices and {len(city_faces_inside_reindexed)} faces.")

# ----------------- OUTSIDE MESH -----------------
city_faces_outside = city_faces_full[mask_outside]
unique_vertices_outside = np.unique(city_faces_outside)
city_vertices_outside = city_vertices_full[unique_vertices_outside]

# Reindex
vertex_reindex_outside = {old: new for new, old in enumerate(unique_vertices_outside)}
city_faces_outside_reindexed = np.vectorize(vertex_reindex_outside.get)(city_faces_outside)

print(f"Outside mesh has {len(city_vertices_outside)} vertices and {len(city_faces_outside_reindexed)} faces.")

# --- Shared plot helpers precomputed here ---
bbox_x = [rx_tx_bbox_min[0], rx_tx_bbox_max[0]]
bbox_y = [rx_tx_bbox_min[1], rx_tx_bbox_max[1]]
z_level = float(np.mean(city_vertices_full[:, 2]))

Original mesh has 2564689 vertices and 3170984 faces.
Full city mesh bounding box:
Min: [0. 0. 0.]
Max: [6862.96875  7557.5       264.600006]
Rx/Tx bounding box (XY only):
Min (XY): [3035.3125 2217.5   ]
Max (XY): [6180.3125 5947.5   ]
Inside mesh has 829663 vertices and 1024496 faces.
Outside mesh has 1735576 vertices and 2146488 faces.


In [7]:
# --- Recover original coordinates for inside vertices ---
utm_x_min_inside_bbox = rx_tx_bbox_min[0] + xmin
utm_x_max_inside_bbox = rx_tx_bbox_max[0] + xmin
utm_y_min_inside_bbox = rx_tx_bbox_min[1] + ymin
utm_y_max_inside_bbox = rx_tx_bbox_max[1] + ymin
print("Inside mesh UTM bounding box:")
print(f"X min: {utm_x_min_inside_bbox}, X max: {utm_x_max_inside_bbox}")
print(f"Y min: {utm_y_min_inside_bbox}, Y max: {utm_y_max_inside_bbox}")

Inside mesh UTM bounding box:
X min: 387627.5, X max: 390772.5
Y min: 3947012.5, Y max: 3950742.5


In [8]:
from pyproj import Proj, Transformer

# Define UTM projection (zone 54N)
utm_proj = Proj(proj='utm', zone=54, ellps='WGS84', south=False)
wgs84_proj = Proj(proj='latlong', datum='WGS84')

# Or use Transformer (better)
transformer = Transformer.from_crs(f"+proj=utm +zone=54 +datum=WGS84 +units=m +no_defs",
                                   "EPSG:4326", always_xy=True)

# Coordinates
x1, y1 = utm_x_min_inside_bbox, utm_y_min_inside_bbox
x2, y2 = utm_x_max_inside_bbox, utm_y_max_inside_bbox

# Convert
x_min_inside_lon1, x_min_inside_lat1 = transformer.transform(x1, y1)
x_min_inside_lon2, x_min_inside_lat2 = transformer.transform(x2, y2)

print(f"Point 1: Latitude {x_min_inside_lat1}, Longitude {x_min_inside_lon1}")
print(f"Point 2: Latitude {x_min_inside_lat2}, Longitude {x_min_inside_lon2}")


Point 1: Latitude 35.66057825471421, Longitude 139.7585143007345
Point 2: Latitude 35.694555217482815, Longitude 139.7927467148492


In [9]:
from pyproj import Transformer
import numpy as np

# --- 1. Compute current extent in meters ---
extent_x = abs(utm_x_max_inside_bbox - utm_x_min_inside_bbox)
extent_y = abs(utm_y_max_inside_bbox - utm_y_min_inside_bbox)
print(f"Original extent (meters): East-West = {extent_x:.2f}, North-South = {extent_y:.2f}")

# --- 2. Extend bounding box by 200m in all directions ---
extension = 500  # meters
utm_x_min_extended = utm_x_min_inside_bbox - extension
utm_x_max_extended = utm_x_max_inside_bbox + extension
utm_y_min_extended = utm_y_min_inside_bbox - extension
utm_y_max_extended = utm_y_max_inside_bbox + extension

# --- Convert extended UTM corners to lat/lon ---
transformer = Transformer.from_crs(
    f"+proj=utm +zone=54 +datum=WGS84 +units=m +no_defs",
    "EPSG:4326",
    always_xy=True
)

lon_min_extended, lat_min_extended = transformer.transform(utm_x_min_extended, utm_y_min_extended)
lon_max_extended, lat_max_extended = transformer.transform(utm_x_max_extended, utm_y_max_extended)

print("\nExtended bounding box (UTM):")
print(f"X min: {utm_x_min_extended}, X max: {utm_x_max_extended}")
print(f"Y min: {utm_y_min_extended}, Y max: {utm_y_max_extended}")

print("\nExtended bounding box (Lat/Lon):")
print(f"Point 1 (min corner): Latitude {lat_min_extended}, Longitude {lon_min_extended}")
print(f"Point 2 (max corner): Latitude {lat_max_extended}, Longitude {lon_max_extended}")

# --- 3. Compute new extent in meters ---
extent_x_extended = abs(utm_x_max_extended - utm_x_min_extended)
extent_y_extended = abs(utm_y_max_extended - utm_y_min_extended)
print(f"\nExtended extent (meters): East-West = {extent_x_extended:.2f}, North-South = {extent_y_extended:.2f}")

Original extent (meters): East-West = 3145.00, North-South = 3730.00

Extended bounding box (UTM):
X min: 387127.5, X max: 391272.5
Y min: 3946512.5, Y max: 3951242.5

Extended bounding box (Lat/Lon):
Point 1 (min corner): Latitude 35.65601402610405, Longitude 139.75306157263827
Point 2 (max corner): Latitude 35.699117715538016, Longitude 139.79820431501943

Extended extent (meters): East-West = 4145.00, North-South = 4730.00


In [10]:
# PREP CELL (no plotting) ---------------------------------------------
import numpy as np

# --- Full city mesh (shifted) ---
city_vertices_full = shifted_vertices
city_faces_full = faces

# --- Compute extended bounding boxes (shifted) ---
utm_x_min_extended_shifted = utm_x_min_extended - xmin
utm_x_max_extended_shifted = utm_x_max_extended - xmin
utm_y_min_extended_shifted = utm_y_min_extended - ymin
utm_y_max_extended_shifted = utm_y_max_extended - ymin

# --- For each face, get its vertices ---
face_vertices_full = city_vertices_full[city_faces_full]  # shape: (num_faces, 3, 3)

# --- Mask faces inside extended bounding box (shifted) ---
mask_extended = np.any(
    (face_vertices_full[:, :, 0] >= utm_x_min_extended_shifted) &
    (face_vertices_full[:, :, 0] <= utm_x_max_extended_shifted) &
    (face_vertices_full[:, :, 1] >= utm_y_min_extended_shifted) &
    (face_vertices_full[:, :, 1] <= utm_y_max_extended_shifted),
    axis=1
)
faces_extended = city_faces_full[mask_extended]
print(f"Faces inside extended bounding box: {len(faces_extended)}")

# --- Mask faces inside original bounding box (subset of extended) ---
mask_inside = np.any(
    (face_vertices_full[:, :, 0] >= rx_tx_bbox_min[0]) &
    (face_vertices_full[:, :, 0] <= rx_tx_bbox_max[0]) &
    (face_vertices_full[:, :, 1] >= rx_tx_bbox_min[1]) &
    (face_vertices_full[:, :, 1] <= rx_tx_bbox_max[1]),
    axis=1
)
faces_red = city_faces_full[mask_inside & mask_extended]  # inside original bbox

# faces_grey = (outside original, inside extended)
faces_grey = np.setdiff1d(
    faces_extended.view([('', faces_extended.dtype)] * faces_extended.shape[1]),
    faces_red.view([('', faces_red.dtype)] * faces_red.shape[1])
).view(faces_extended.dtype).reshape(-1, 3)

print(f"Grey faces (outside original, inside extended): {len(faces_grey)}")
print(f"Red faces (inside original bounding box): {len(faces_red)}")

# --- Reindex vertices for plotting ---
unique_vertices_grey = np.unique(faces_grey)
unique_vertices_red  = np.unique(faces_red)

vertex_map_grey = {old: new for new, old in enumerate(unique_vertices_grey)}
faces_grey_reindexed = np.vectorize(vertex_map_grey.get)(faces_grey)

vertex_map_red = {old: new for new, old in enumerate(unique_vertices_red)}
faces_red_reindexed = np.vectorize(vertex_map_red.get)(faces_red)

vertices_grey_shifted = city_vertices_full[unique_vertices_grey]
vertices_red_shifted  = city_vertices_full[unique_vertices_red]

print(f"Unique vertices for grey faces: {len(unique_vertices_grey)}")
print(f"Unique vertices for red faces: {len(unique_vertices_red)}")

# --- Height for plotting bounding boxes ---
z_level = float(np.mean(vertices_grey_shifted[:, 2])) if len(vertices_grey_shifted) else 0.0

# --- Precompute simple bbox arrays used by the plot cell ---
bbox_x     = [rx_tx_bbox_min[0], rx_tx_bbox_max[0]]     # fixed small typo from earlier line break
bbox_y     = [rx_tx_bbox_min[1], rx_tx_bbox_max[1]]
bbox_x_ext = [utm_x_min_extended_shifted, utm_x_max_extended_shifted]
bbox_y_ext = [utm_y_min_extended_shifted, utm_y_max_extended_shifted]

Faces inside extended bounding box: 1724880
Grey faces (outside original, inside extended): 405764
Red faces (inside original bounding box): 1024496
Unique vertices for grey faces: 567549
Unique vertices for red faces: 829663


In [11]:
# --- Faces inside the extended bbox (you already have mask_extended) ---
faces_extended = city_faces_full[mask_extended]               # (K, 3) int

# --- Unique vertex indices used by those faces ---
unique_vertices_extended = np.unique(faces_extended.ravel())  # (U,)

# --- Build an old->new index map and reindex faces ---
new_index = -np.ones(city_vertices_full.shape[0], dtype=int)
new_index[unique_vertices_extended] = np.arange(unique_vertices_extended.size)

city_faces_extended_shifted = new_index[faces_extended]       # (K, 3) int (reindexed)

# --- Slice vertex array to only used vertices (shifted coords you already have) ---
city_vertices_extended_shifted = city_vertices_full[unique_vertices_extended]  # (U, 3) float

print("city_vertices_extended_shifted:", city_vertices_extended_shifted.shape)
print("city_faces_extended_shifted:", city_faces_extended_shifted.shape)

city_vertices_extended_shifted: (1396662, 3)
city_faces_extended_shifted: (1724880, 3)


In [12]:
import numpy as np
from pathlib import Path

OUT = Path("./portable_mesh.npz")
np.savez_compressed(
    OUT,
    V=city_vertices_extended_shifted.astype(float),
    F=city_faces_extended_shifted.astype(int),
    xmin=float(xmin),
    ymin=float(ymin),
)
print("Saved ->", OUT.resolve())

Saved -> /home/mkrishne/PL_competition/portable_mesh.npz
